In [1]:
from mpmath import mp, mpf, findroot, bessely
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

In [2]:
from sage.all import *

R = LaurentSeriesRing(RealField(300), 'x', default_prec=300)
x = R.gen()
N = 16  # Number of terms (adjust as needed)
gamma = RealField(300)(euler_gamma)
d2 = RealField(300)(2)
pi = RealField(300).pi()
log2 = RealField(300)(2).log()

def j_series(n, x):
    return sum([(-1)**m * (x/2)**(ZZ(n) + ZZ(2)*ZZ(m)) / (ZZ(m).factorial() * (ZZ(m) + ZZ(n)).factorial()) for m in range(N)])

J1_series = j_series(1, x)

def harmony(m):
    return sum(RealField(300)(1)/RealField(300)(k) for k in range(1, m+1))

def z_series(x):
    return sum([(-1)**m * (x)**(ZZ(2)*ZZ(m)) / (ZZ(2)**(2*m) * ZZ(m).factorial() * (ZZ(m) - ZZ(1)).factorial()) * (harmony(m) + harmony(m - 1)) for m in range(1, N)])

W1 = d2/pi * J1_series
Z1 = -(d2/(x*pi) * z_series(x) + d2/pi * gamma * J1_series(x) - d2/pi * log2 * J1_series(x))

def y1_full(x):
    return d2/pi * (J1_series(x) * x.log() - 1/x * ( 1 - z_series(x)) + (gamma - log2) * J1_series(x))

def y1_full_expanded(x):
    return d2/pi * J1_series(x) * x.log() - d2/(x*pi) + d2/(x*pi) * z_series(x) + d2/pi * gamma * J1_series(x) - d2/pi * log2 * J1_series(x)

# print("W0(x) =", W0)
# print("Z0(x) =", Z0)

terms = 30

from mpmath import mp, bessely, taylor

def Y1_approx(val):
    # Substitute numeric value and convert to high precision real
    w = W1.truncate(terms)(val)
    z = Z1.truncate(terms)(val)
    return RealField(107)(w) * RealField(107)(val).log() - RealField(107)(z) - RealField(107)(2)/(pi * val)

mp.prec = 120

print("W = ", RealField(64)(W1.truncate(terms)(0.89357696627916749)))
print("Z = ", RealField(64)(Z1.truncate(terms)(0.89357696627916749)))
print("LOG = ", RealField(110)(0.89357696627916749).log())
print_double_double("", RealField(110)(2).log())
print("Y1 approx = ", Y1_approx(RealField(100)(0.89357696627916749)))
print("Y1 full = ", y1_full(RealField(100)(0.89357696627916749)))
print("Y1 full expanded = ", y1_full_expanded(RealField(100)(0.89357696627916749)))
print("Y1 = ", bessely(1, 0.89357696627916749))

print(W1.truncate(10))

def print_series(z, N):
    for i in range(N):
        coeff = z[i]
        if coeff != 0:
            # print(double_to_hex(RealField(300)(coeff)) + ",")
            print_double_double("", RealField(300)(coeff))

print_series(Z1, 30)

# print_double_double("2/PI", -d2/pi)

W =  0.256973952516943105
Z =  0.138065597292926445
LOG =  -0.11252280788079415479644853102767
(0x3c7abc9e3b39803f, 0x3fe62e42fefa39ef),
Y1 approx =  -0.87942080249719479722972442056
Y1 full =  -0.87942080249719479722972442056
Y1 full expanded =  -0.87942080249719479722972442056
Y1 =  -0.87942080249719479722972442056326109
0.318309886183790671537767526745028724068919291480912897495334688117793595268453070180227606*x + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 - 0.0397887357729738339422209408431285905086149114351141121869168360147241994085566337725284507*x^3 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 + 0.00165786399054057641425920586846369127119228797646308800778820150061350830868985974052201878*x^5 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 - 0.000034538833136262008630400122259660234816505999509647666828920864596114756431

In [2]:
R120 = RealField(120)

zeros = []

mp.prec = 160

step = mpf("0.1")
epsilon = mpf("1e-35")
x = mpf("1.25")

previous_zero = R120(0)
y1_zeros = []

while x < mpf("52.0"):
    f1 = bessely(1, x)
    f2 = bessely(1, x + step)
    if f1 * f2 < 0:
        zero = findroot(lambda t: bessely(1, t), (x, x + step), solver='secant', tol=mp.mpf("1e-41"))
        previous_zero = zero
        y1_zeros.append(zero)
    if previous_zero is not None and abs(x - mpf(f'{round(x)}')) < epsilon:
        zeros.append(previous_zero)
    x += step

y1_extrema = []

x = mpf("1.25")
while x < mpf("52.0"):
    d1 = mp.diff(lambda t: bessely(1, t), x)
    d2 = mp.diff(lambda t: bessely(1, t), x + step)
    if d1 * d2 < 0:
        extremum = findroot(lambda t: mp.diff(lambda u: bessely(1, u), t), (x, x + step), solver='secant', tol=mp.mpf("1e-41"))
        y1_extrema.append(extremum)
    x += step

# Print results
for i, z in enumerate(y1_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of Y1(x):")
for e in y1_extrema:
    print(f"nExtrema: {e}")

y1_zeros.extend(y1_extrema)

y1_zeros = sorted(y1_zeros)

# Print results
for i, z in enumerate(y1_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

# print(f"pub(crate) static Y1_ZEROS: [(u64, u64); {len(y1_zeros)}] = [")
# print(f"(0x0, 0x0),")
# for z in y1_zeros:
#     k = split_double_double(z)
#     hi = double_to_hex(k[1])
#     lo = double_to_hex(k[0])
#     print(f"({lo}, {hi}),")
    
# print("];")

print(f"pub(crate) static Y1_ZEROS_RATIONAL128: [DyadicFloat128; {len(y1_zeros)}] = [")
print(f"DyadicFloat128 {{ sign: DyadicSign::Pos, exponent: 0, mantissa: 0x0u128, }},")
for z in y1_zeros:
    print_dyadic(z)
    
print("];")

Zero 1: x ≈ 2.1971413260310170351490335626989662730530183315
Zero 2: x ≈ 5.429681040794135132772005190852584196583757476
Zero 3: x ≈ 8.596005868331168926429606180163967851102921567
Zero 4: x ≈ 11.749154830839881243399421939922350714301165983
Zero 5: x ≈ 14.897442128336725378844819156429870879807150631
Zero 6: x ≈ 18.043402276727855564304555507889508902163088325
Zero 7: x ≈ 21.188068934142213016142481528685423196935024605
Zero 8: x ≈ 24.33194257135691203599294405185012965141433334
Zero 9: x ≈ 27.475294980449223512212285525410668235700897307
Zero 10: x ≈ 30.618286491641114715761625696447448310277939571
Zero 11: x ≈ 33.761017796109325692471759911249650993879821496
Zero 12: x ≈ 36.903555316142950053588159309844303659060544565
Zero 13: x ≈ 40.045944640266876089064214537312936172020628336
Zero 14: x ≈ 43.18821809739321126847753320012759720745635785
Zero 15: x ≈ 46.330399250701686587724463565243621670014635651
Zero 16: x ≈ 49.472505679924095824128003887609267273294894412
Extrema (peaks/valley

In [10]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = 1.35 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.05 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.05 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.05
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.05  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(y1_zeros)
print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 250;
bessel_y1 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_y1(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RR(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

degree = 21

print(f"pub(crate) static Y1F_COEFFS: [[u64;{degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print("[")
    for c in coeffs:
        print(double_to_hex(c) + ",")
    print("],")
print("];")

[(-0.84714, 0.79294, 2.1971413260310170351490335626990), (-0.79294, 0.92333, 3.6830228565851776998989672428340), (-0.92333, 0.80591, 5.4296810407941351327720051908526), (-0.80591, 0.87725, 6.9414999536541756557519437170770), (-0.87725, 0.81370, 8.5960058683311689264296061801640), (-0.81370, 0.86288, 10.123404655436613079787752723081), (-0.86288, 0.81830, 11.749154830839881243399421939922), (-0.81830, 0.85584, 13.285758156782854437262071956503), (-0.85584, 0.82131, 14.897442128336725378844819156430), (-0.82131, 0.85167, 16.440058007293281606967213074369), (-0.85167, 0.82342, 18.043402276727855564304555507890), (-0.82342, 0.84892, 19.590241756629494709149314724222), (-0.84892, 0.82498, 21.188068934142213016142481528685), (-0.82498, 0.84695, 22.738034717396327495094571334537), (-0.84695, 0.82619, 24.331942571356912035992944051850), (-0.82619, 0.84549, 25.884314618788867398121669515049), (-0.84549, 0.82714, 27.475294980449223512212285525411), (-0.82714, 0.84436, 29.029575819372535446204461

In [60]:
mp.prec = 60
terms = 29
print(f"pub(crate) static Y1F_COEFFS: [[u64; {terms}]; {len(y1_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly, n):
    print("[")
    for i in range(0, n):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(y1_zeros)):
    k_range = y1_zeros[i]
    range_diff = k_range - prev_zero

    x0 = mp.mpf(k_range)
    from mpmath import mp, bessely, taylor
    poly = taylor(lambda val: bessely(1, val), x0, terms)
    # print(poly)
    print_taylor_coeffsf(poly, terms)
    prev_zero = y1_zeros[i]

print("];")

pub(crate) static Y1F_COEFFS: [[u64; 29]; 32] = [
[
0x0000000000000000,
0x3fe0aa48442f014b,
0xbfbe56f82217b8f1,
0xbfa0d2af4e932386,
0xbf73a6dec37290aa,
0x3f7e671c7d1196fd,
0xbf65429dc5a45612,
0x3f517ab4afac0735,
0xbf40b2d877d5a29c,
0x3f2eea7bbb907646,
0xbf1c3fae3b0ad706,
0x3f09d174c9ccecd9,
0xbef7956ad3fe9783,
0x3ee5865d11ebb14d,
0xbed3a27f1fd4ac6e,
0x3ec1e742c1cf6acb,
0xbeb051d134cb0a72,
0x3e9dbea846629267,
0xbe8b1a1682f5eb1f,
0x3e78b0b08d8ccddf,
0xbe667d8a7cc18981,
0x3e547c0e174bc402,
0xbe42a7fb50d94c09,
0x3e30fd661ade45f0,
0xbe1ef1655b283bca,
0x3e0c2d21fe733c5f,
0xbdf9a7efb711afac,
0x3de75c3fc5194fe2,
0xbdd544fd359e8d1a,
],
[
0x3fdaabb4011ed330,
0x0000000000000000,
0xbfc8b45babe797ba,
0x3f8e147099a6f0ea,
0x3f88c5af1eeb2d6b,
0xbf4133fa47da52e2,
0xbf3bf8af93ff0b8c,
0x3f021d64be62ccf9,
0x3eb44d2ce67b2d18,
0x3eb14c3ab2ad79cc,
0xbe9b8eee52e1ce01,
0x3e7a85f1a878746a,
0xbe5bf86b3c89849b,
0x3e3f01920a1ecb6b,
0xbe20fac544c5a674,
0x3e02818841d3a289,
0xbde42b5ceddd3872,
0x3dc5f9472a38633d,
0xb

In [11]:
def compute_intervals(zeros):
    intervals = []
    for i in range(0, len(zeros)):
        if i == 0:
            a = 1.4 - zeros[i]
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.03 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        elif i + 1 > len(zeros) - 1:
            a = (zeros[i - 1] + zeros[i]) / 2 - 0.03 - zeros[i]
            b = (zeros[i]) + 0.83 + 0.03 - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
        else:
            a = (zeros[i - 1] + zeros[i]) / 2 - zeros[i] - 0.03
            b = (zeros[i] + zeros[i + 1]) / 2 + 0.03  - zeros[i]
            intervals.append((RealField(18)(a), RealField(18)(b), RealField(110)(zeros[i])))
    return intervals

intervals = compute_intervals(y1_zeros)
print(intervals)

def build_sollya_script(a, b, zero, deg):
    return f"""
prec = 200;
bessel_y1 = library("/Users/radzivon/RustroverProjects/pxfm/notes/bessel_sollya/cmake-build-release/libbessel_sollya.dylib");
f = bessel_y1(x + {zero});
d = [{a}, {b}];
pf = remez(f, {deg}, d);
for i from 0 to degree(pf) do {{
    write(coeff(pf, i)) >> "coefficients.txt";
    write("\\n") >> "coefficients.txt";
}};
"""

def load_coefficients(filename):
    with open(filename, "r") as f:
        return [RealField(500)(line.strip()) for line in f if line.strip()]

def call_sollya_on_interval(a, b, zero, degree=12):
    sollya_script = build_sollya_script(a, b, zero, degree)
    with open("tmp_interval.sollya", "w") as f:
        f.write(sollya_script)
    import subprocess
    if os.path.exists("coefficients.txt"):
        os.remove("coefficients.txt")
    try:
        result = subprocess.run(
            ["sollya", "tmp_interval.sollya"],
            check=True,
            capture_output=True,
            text=True
        )
    except subprocess.CalledProcessError as e:
        return

def print_remez_coeffs(poly):
    print("[")
    for i in range(len(poly)):
        coeff = poly[i]
        print_double_double("", coeff)
    print("],")

degree = 27

print(f"pub(crate) static Y1_COEFFS: [[(u64, u64); {degree + 1}]; {len(intervals)}] = [")
for i in range(0, len(intervals)):
    interval = intervals[i]
    call_sollya_on_interval(interval[0], interval[1], interval[2], degree)
    coeffs = load_coefficients(f"coefficients.txt")
    print_remez_coeffs(coeffs)
print("];")

[(-0.79714, 0.77294, 2.1971413260310170351490335626990), (-0.77294, 0.90333, 3.6830228565851776998989672428340), (-0.90333, 0.78591, 5.4296810407941351327720051908526), (-0.78591, 0.85725, 6.9414999536541756557519437170770), (-0.85725, 0.79370, 8.5960058683311689264296061801640), (-0.79370, 0.84288, 10.123404655436613079787752723081), (-0.84288, 0.79830, 11.749154830839881243399421939922), (-0.79830, 0.83584, 13.285758156782854437262071956503), (-0.83584, 0.80131, 14.897442128336725378844819156430), (-0.80131, 0.83167, 16.440058007293281606967213074369), (-0.83167, 0.80342, 18.043402276727855564304555507890), (-0.80342, 0.82891, 19.590241756629494709149314724222), (-0.82891, 0.80498, 21.188068934142213016142481528685), (-0.80498, 0.82695, 22.738034717396327495094571334537), (-0.82695, 0.80619, 24.331942571356912035992944051850), (-0.80619, 0.82549, 25.884314618788867398121669515049), (-0.82549, 0.80714, 27.475294980449223512212285525411), (-0.80714, 0.82436, 29.029575819372535446204461

In [8]:
# Taylor series for f64
# mp.prec = 110
# terms = 50
# print(f"pub(crate) static Y1_COEFFS: [[(u64, u64); {terms}]; {len(y1_zeros)}] = [")

# def get_constant_term(poly, y):
#     for term in poly.operands():
#         if term.is_constant():
#             return term

# def print_taylor_coeffs(poly, n):
#     print("[")
#     for i in range(0, n):
#         coeff = poly[i]
#         print_double_double("", RealField(110)(coeff))
#         # print(f"{double_to_hex(coeff)},")
#     print("],")

# prev_zero = 0

# for i in range(0, len(y1_zeros)):
#     k_range = y1_zeros[i]
#     range_diff = k_range - prev_zero

#     x0 = mp.mpf(k_range)
#     from mpmath import mp, bessely, taylor
#     poly = taylor(lambda val: bessely(1, val), x0, terms)
#     # print(poly)
#     print_taylor_coeffs(poly, terms)
#     prev_zero = y1_zeros[i]

# print("];")

mp.prec = 140
terms = 28
print(f"pub(crate) static Y1_COEFFS_RATIONAL128: [[DyadicFloat128; {terms}]; {len(y1_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_dyadic_cef(poly, n):
    print("[")
    for i in range(0, n):
        coeff = poly[i]
        print_dyadic(coeff)
    print("],")

prev_zero = 0

for i in range(0, len(y1_zeros)):
    k_range = y1_zeros[i]
    range_diff = k_range - prev_zero

    x0 = mp.mpf(k_range)
    from mpmath import mp, bessely, taylor, chebyfit
    poly = taylor(lambda val: bessely(1, val), x0, terms)
    print_taylor_dyadic_cef(poly, terms)
    prev_zero = y1_zeros[i]

print("];")

pub(crate) static Y1_COEFFS_RATIONAL128: [[DyadicFloat128; 28]; 32] = [
[
0.0000000000000000000000000000000
0.5207864124022675111071507369263
-0.1185145457490966085093808356841
-0.03285739740528640971163961951380
-0.004797811670105437566231233314102
0.007422553332707468527492873466167
-0.002595241688264201551532985498391
0.001066853000054779797186408785340
-0.0005096013043558058528087694943149
0.0002358699837551760903508966274672
-0.0001077604381503859994292656022081
0.00004924428615146200596913648038685
-0.00002249113208532090530010863122691
0.00001026385194565688518638531309521
-4.681289735668145256341454463729e-6
2.134247110136465624787394952351e-6
-9.727238704937524529074521878047e-7
4.432313914870946156980474440609e-7
-2.019249349774460978612801234777e-7
9.197814011541545681914675925056e-8
-4.189162403145422948589101469374e-8
1.907776110954049084206358447202e-8
-8.687460108203660479127469600523e-9
3.955755207677639513003680567408e-9
-1.801116905471169729080040808407e-9
8.200382568

KeyboardInterrupt: 